In this notebook, we will train a network to perform group composition on the abstract group S5, and then reverse engineer the algorithm learned. This notebook should be fairly self contained, and uses similar code to that in the paper. Full code for the paper is available at ...

# Setup


In [8]:
# Imports
import os
if "cd" not in globals():
    os.chdir("../")
    cd = True
import torch
import torch.nn as nn
import numpy as np
import math
from tqdm import tqdm
from transformer_lens.hook_points import HookPoint, HookedRootModule
from sympy.combinatorics.named_groups import SymmetricGroup as SymPySymmetricGroup
from sympy.combinatorics import Permutation, PermutationGroup
from utils.plotting import *

In [9]:
SEED = 2

Let's first create a class to contain all the information about our group. We include methods 
* to convert between permutations and their indices that we use to one-hot-encode them. 
* to compose permutations, based on their index
* to calculate the order of a permutation
* to calculate the signature of a permutation
* to cache all signatures of the group elements
* to calculate the inverse of a permutation
* to cache all inverses of group elements
* to calculate the full multiplication table of the group
* to return all pairs of permutations, and their product

In [10]:
class SymmetricGroup():
    """
    Class for the symmetric group of order index.
    """
    def __init__(self, index, init_all=False):
        """
        Initialise the group. Optionally calculate all other tensors needed to track metrics.

        Args:
            index (int): Index of group in family of symmetric groups.
            init_all (bool, optional): If false, only calculate what is required to train. If true, calculate all other tensors needed to track metrics. Defaults to False.
        """

        # build class on top of sympy
        self.G = SymPySymmetricGroup(index)
        self.index = index
        self.order = math.factorial(index)
        self.acronym = 'S'
        self.multiplication_table = self.compute_multiplication_table()
        
        # hacky method to find the index of the identity element 
        self.identity = [i for i in range(self.order) if self.idx_to_perm(i).order() == 1][0]
        
        self.inverses = self.compute_inverses()

        # compute the signatures of the elements
        self.signatures = self.compute_signatures()

        self.all_data, _ = self.get_all_data()
        self.all_data = self.all_data[:, :2]


    def idx_to_perm(self, x):
        """
        Convert an index to a permutation.

        Args:
            x (int): index of element in group

        Returns:
            Permutation: permutation object from sympy
        """
        return self.G._elements[x]

    def perm_to_idx(self, perm):
        """
        Converts a permutation to an index.

        Args:
            perm (Permutation): permutation object from sympy

        Returns:
            int: index of element in group
        """
        return self.G._elements.index(perm)

    def compose(self, x, y):
        """
        Compose elements of the group by converting to permutations, composing, and converting back.

        Args:
            x (int): left index
            y (int): right index

        Returns:
            int: index of composition
        """
        return self.perm_to_idx(self.idx_to_perm(x) * self.idx_to_perm(y))

    def perm_order(self, x):
        """
        Gets the order of a permutation.

        Args:
            x (int): index of element

        Returns:
            int: order of permutation
        """
        return self.idx_to_perm(x).order()

    def signature(self, x):
        """
        Gets the signature of a permutation.

        Args:
            x (int): index of element

        Returns:
            int: Integer \in {0, 1} representing the signature of the permutation.
        """
        return self.idx_to_perm(x).signature()
    
    def compute_signatures(self):
        """
        Compute and store the signature of each element in the group.

        Returns:
            torch.tensor: tensor of signatures
        """
        signatures = torch.tensor([self.signature(i) for i in range(self.order)]).cuda()
        return signatures
    
    def inverse(self, x):
        """
        Compute the inverse of an element of the group.

        Args:
            x (int): Index of element to inverse

        Returns:
            int: Index of inverse element
        """
        return (self.multiplication_table[x, :] == self.identity).nonzero().item()

    def compute_inverses(self):
        inverses = torch.zeros(self.order, dtype=torch.int64)
        for i in range(self.order):
            inverses[i] = self.inverse(i)
        return inverses
    
    def compute_multiplication_table(self):
        """
        Compute the multiplication table of the group. Caches/loads from file if possible.
        """
        print('Computing multiplication table...')
        table = torch.zeros((self.order, self.order), dtype=torch.int64).cuda()
        for i in tqdm(range(self.order)):
            for j in range(self.order):
                table[i, j] = self.compose(i, j)
        return table

    def get_all_data(self, shuffle_seed=False):
        """
        Get's all data and labels for the pairwise composition task.

        Args:
            shuffle_seed (bool, optional): Shuffle data for training. Defaults to False.

        Returns:
            torch.tensor: Tensor of shape (order*order, 3) where each row is (x, y, x*y).
        """
        data=torch.zeros((self.order*self.order, 3), dtype=torch.int64)
        shuffled_indices = None
        for i in range(self.order):
            for j in range(self.order):
                data[i*self.order+j, 0] = i
                data[i*self.order+j, 1] = j
                data[i*self.order+j, 2] = self.multiplication_table[i, j]
        if shuffle_seed:
            torch.manual_seed(shuffle_seed) 
            shuffled_indices = torch.randperm(self.order*self.order)
            data = data[shuffled_indices]
        return data.cuda(), shuffled_indices

In [11]:
group = SymmetricGroup(5)

Computing multiplication table...


100%|██████████| 120/120 [00:14<00:00,  8.42it/s]


The model architecture is an MLP as described in the paper. We subclass the HookedRootModule class from the transformer_lens library to easily cache activations on the forward pass.

In [12]:
model_cfg = {    
    "layers": {
        "embed_dim": 256,
        "hidden_dim": 128
    }
}

class OneLayerMLP(HookedRootModule):
    """ 
    A one layer MLP. W_x and W_y are embedding layers, whose outputs are concatenated and fed into a hidden layer. The result is unembedded by W_U.
    """
    def __init__(self, layers, n, seed=0):
        # embed_dim: dimension of the embedding
        # hidden : hidden dimension size
        # n : group order
        super().__init__()
        torch.manual_seed(seed)

        self.embed_dim = layers['embed_dim']
        hidden = layers['hidden_dim']

        # xavier initialise parameters
        self.W_x = nn.Parameter(torch.randn(n, self.embed_dim)/np.sqrt(self.embed_dim))
        self.W_y = nn.Parameter(torch.randn(n, self.embed_dim)/np.sqrt(self.embed_dim))
        self.W = nn.Parameter(torch.randn(2*self.embed_dim, hidden)/np.sqrt(2*self.embed_dim))
        self.relu = nn.ReLU()
        self.W_U = nn.Parameter(torch.randn(hidden, n)/np.sqrt(hidden))

        # hookpoints
        self.embed_stack = HookPoint()
        self.hidden = HookPoint()

        # We need to call the setup function of HookedRootModule to build an 
        # internal dictionary of modules and hooks, and to give each hook a name
        super().setup()

    def forward(self, data):
        x = data[:, 0] # (batch)
        half_x_embed = self.W_x[x] # (batch, embed_dim)
        y = data[:, 1] # (batch)
        half_y_embed = self.W_y[y] # (batch, embed_dim)
        embed_stack = self.embed_stack(torch.hstack((half_x_embed, half_y_embed))) # (batch, 2*embed_dim)
        hidden = self.hidden(self.relu(embed_stack @ self.W)) # (batch, hidden)
        out = hidden @ self.W_U # (batch, n)
        return out

def loss_fn(logits, labels):
    logits = logits.to(torch.float64)
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[:, None])[:, 0]
    return -correct_log_probs.mean()

def get_accuracy(logits, labels):
    """
    Compute accuracy of model.

    Args:
        logits (torch.tensor): (batch, group.order) tensor of logits
        labels (torch.tensor): (batch) tensor of labels

    Returns:
        float: accuracy
    """
    return ((logits.argmax(1)==labels).sum()/len(labels)).item()

model = OneLayerMLP(model_cfg['layers'], group.order).cuda()

To generate our train/test split, we just take a fixed proportion of the entire multiplication table of the group at random. We'll use a ratio of 0.4 here.

In [13]:
def generate_train_test_data(group, frac_train, seed=False):
    """
    Generate train and test data from a group's all data.
    """
    data, shuffled_indices = group.get_all_data(seed)
    train_size = int(frac_train*data.shape[0])
    train = data[:train_size]
    test = data[train_size:]
    train_data = train[:, :2]
    train_labels = train[:, 2]
    test_data = test[:, :2]
    test_labels = test[:, 2]
    return train_data, test_data, train_labels, test_labels, shuffled_indices



# Training

In [14]:
training_cfg = {
    "lr": 0.001,
    "betas": (0.9, 0.98),
    "weight_decay": 1,
    "num_epochs": 75000,
    "frac_train": 0.4,
}
train_data, test_data, train_labels, test_labels, shuffled_indices = generate_train_test_data(group, training_cfg["frac_train"], seed=SEED)
optimizer = torch.optim.AdamW(model.parameters(), lr=training_cfg["lr"], betas=training_cfg["betas"], weight_decay=training_cfg["weight_decay"])

In [15]:
print(train_data)

tensor([[ 65,  48],
        [ 85,  94],
        [104,  89],
        ...,
        [ 92,  13],
        [ 75,  94],
        [  7,  80]], device='cuda:0')


In [16]:
train_losses = []
test_losses = []
train_accs = []
test_accs = []

for epoch in tqdm(range(training_cfg["num_epochs"])):
    train_logits = model(train_data)
    train_loss = loss_fn(train_logits, train_labels)
    train_loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if epoch % 1000 == 0:
        with torch.inference_mode():
            test_logits = model(test_data)
            test_loss = loss_fn(test_logits, test_labels)
            train_acc = get_accuracy(train_logits, train_labels)
            test_acc = get_accuracy(test_logits, test_labels)
            print(f"Epoch {epoch}: Train Loss: {train_loss}, Test Loss: {test_loss}, Train Accuracy: {train_acc}, Test Accuracy: {test_acc}")
        train_losses.append(train_loss.item())
        test_losses.append(test_loss.item())
        train_accs.append(train_acc)
        test_accs.append(test_acc)





  0%|          | 50/75000 [00:00<09:03, 137.93it/s]

Epoch 0: Train Loss: 4.788640449718693, Test Loss: 4.789846160721121, Train Accuracy: 0.008854166604578495, Test Accuracy: 0.007986110635101795


  1%|▏         | 1084/75000 [00:02<02:32, 486.01it/s]

Epoch 1000: Train Loss: 0.0014230852654155432, Test Loss: 31.025889312314813, Train Accuracy: 1.0, Test Accuracy: 0.00023148147738538682


  3%|▎         | 2070/75000 [00:04<02:26, 497.34it/s]

...

 97%|█████████▋| 73099/75000 [02:31<00:03, 501.17it/s]

Epoch 73000: Train Loss: 2.1019317401172256e-06, Test Loss: 2.754367274920337e-06, Train Accuracy: 1.0, Test Accuracy: 1.0


 99%|█████████▉| 74068/75000 [02:33<00:01, 503.00it/s]

Epoch 74000: Train Loss: 2.0976692827139747e-06, Test Loss: 2.751178487840493e-06, Train Accuracy: 1.0, Test Accuracy: 1.0


100%|██████████| 75000/75000 [02:35<00:00, 482.26it/s]


Let's plot loss and accuracy. We see the model groks! It initially memorises the training data, and much later on (around epoch 70k) suddenly generalises to the test set.

In [17]:
x = list(range(0, training_cfg["num_epochs"], 1000))
lines([train_losses, test_losses], x=x, labels=["Train Loss", "Test Loss"], xaxis="Epoch", yaxis="Loss", title="Loss vs Epoch", log_y=True)
lines([train_accs, test_accs], x=x, labels=["Train Accuracy", "Test Accuracy"], xaxis="Epoch", yaxis="Accuracy", title="Accuracy vs Epoch")

# Interpretability

Now let's try to interpret the model. To do so, we need to know some things about representation theory. Let's first create a class to hold information about representations. It will share some attributes with the group class. We will subclass from this class to create a class for each representation.

What information do we need, and how do we store it? 

Let $n$ be the order of group $G$. Each representation is a set $n$ $d \times d$ matrices. We store this in our representation objects as tensors of shape $n \times d^2$. 

We also compute what we call an 'orthogonal representation'. I'll explain why this is necessary later on.

Finally we compute the characters $\chi(abc^{-1})$ for each pair of inputs $a,b$ and output $c$. We store this in a tensor of shape $n \times n \times n$. This character tensor is used later to compute logit similarity.

## Defining Representations

In [20]:
class Representation():
    """
    Base class for all representations.
    """

    def __init__(self, compute_rep_params, index, order, multiplication_table, inverses, all_data, group_acronym, irrep=True):
        """
        Initialise the symmetric group representation.

        Args:
            compute_rep_params (tuple): representation specific parameters required for computing the representation
            index (int): group index in family
            order (int): order of the group
            multiplication_table (torch.tensor): square (group.order, group.order) tensor of group multiplication table 
            inverses (torch.tensor): vector of group inverses
            all_data ()
            irrep (Boolean)
        """

        self.index = index
        self.order = order
        self.multiplication_table = multiplication_table
        self.inverses = inverses
        self.all_data = all_data
        self.group_acronym = group_acronym

        # TODO: this is needed to get the dimension of generated representations - think up a better way of doing this
        self.compute_rep_params = compute_rep_params

        self.dim = self.get_rep_dim()

        self.rep = self.compute_rep(*compute_rep_params)

        if irrep:
            self.orth_rep = self.compute_orth_rep(self.rep)
            if self.friendly_name != 'trivial':
                self.logit_trace_tensor_cube = self.compute_logit_trace_tensor_cube()


    def get_rep_dim(self):
        return NotImplementedError

    def compute_rep(self):
        """
        Compute the representation. Must be implemented by child class.

        Raises:
            NotImplementedError
        """
        raise NotImplementedError
    
    def compute_orth_rep(self, rep):
        """
        Use QR decomposition to orthogonalise the representation but retain the subspace spanned by the columns. 

        Args:
            rep (torch.tensor): (group.order, dim^2) tensor of representation

        Returns:
            torch.tensor: (group.order, dim^2) tensor with orthonormal columns en
        """
        
        orth_rep = rep.reshape(self.order, self.dim * self.dim)
        orth_rep = torch.linalg.qr(orth_rep)[0]
        return orth_rep


    def compute_logit_trace_tensor_cube(self):
        """
        Under the hypothesis, the network computes tr(\rho(x)\rho(y)\rho(z^-1)) for some representation \rho.
        This function computes this trace tensor cube for a given representation
        
        Returns:
            torch.tensor: (group.order^3) trace tensor cube
        """
        print(f'Computing trace tensor cube for {self.friendly_name} representation')
        filename = f'utils/cache/{self.group_acronym}{self.index}/{self.group_acronym}{self.index}_{self.friendly_name}_trace_tensor_cube.pt'
        if os.path.exists(filename):
            print('... loading from file')
            t = torch.load(filename)
            return t 
        N = self.all_data.shape[0]
        t = torch.zeros((self.order*self.order, self.order), dtype=torch.float).cuda()
        for i in tqdm(range(N)):
            x = self.all_data[i, 0]
            y = self.all_data[i, 1]
            xy = self.multiplication_table[x, y]
            for z_idx in range(self.order):
                xyz = self.multiplication_table[xy, self.inverses[z_idx]]
                t[i, z_idx] = torch.trace(self.rep[xyz])
        t = t.reshape(self.order, self.order, self.order)
        f = open(filename, 'wb')
        torch.save(t, f)
        return t 

Now we construct all the irreducible representations of S5. One can find a full list [here](https://groupprops.subwiki.org/wiki/Linear_representation_theory_of_symmetric_group:S5). Feel free to skip over this construction if you're not interested in the details.

First, the boring trivial reprsentation. This is one dimensional - that is $d=1$, and all the elements map to the matrix $[1]$. We pass information needed to compute and initialise the representation from the child class to the parent class via `compute_rep_params` and `init_rep_params`.

### Constructing Irreps of S5


In [21]:
class TrivialRepresentation(Representation):
    """
    The trivial representation of the symmetric group.
    """
    def __init__(self, compute_rep_params, init_rep_params):
        """
        Initialise the trivial representation. 

        Args:
            compute_rep_params (list): idx_to_perm function required to compute the representation
            init_rep_params (dict): standard group parameters needed by the representation, including index, order, multiplication_table, inverses, all_data
        """
        self.friendly_name = 'trivial'
        super().__init__(compute_rep_params, **init_rep_params, irrep=True)
    
    def get_rep_dim(self):
        """
        Get the dimension of the representation.

        Returns:
            int: dimension of the representation
        """
        return 1

    def compute_rep(self):
        """
        Compute the trivial representation.

        Args:
            idx_to_perm (function): function to convert an index to a permutation

        Returns:
            torch.tensor: (group.order, 1) tensor of representation
        """
        return torch.ones((self.order, 1, 1), dtype=torch.float).cuda()
    

# we need some data from the group to initialise representations
rep_params = {
            'index': group.index,
            'order': group.order,
            'multiplication_table': group.multiplication_table,
            'inverses': group.inverses,
            'all_data': group.all_data,
            'group_acronym': group.acronym
        }

trivial_rep = TrivialRepresentation(compute_rep_params=[], init_rep_params=rep_params)

Next, the sign representation. This is also 1 dimensional, but the elements map to $[1]$ or $[-1]$ depending on whether the permutation is even or odd.

In [22]:
class SignRepresentation(Representation):
    """
    Initialise the sign representation of the symmetric group.

    """
    def __init__(self, compute_rep_params, init_rep_params):
        """
        Initialise the sign representation. 

        Args:
            compute_rep_params (list): list consisting of the signatures object required to compute the representation
            init_rep_params (dict): standard group parameters needed by the representation, including index, order, multiplication_table, inverses, all_data
        """
        self.friendly_name = 'sign'
        super().__init__(compute_rep_params, **init_rep_params)

    def get_rep_dim(self):
        """
        Get the dimension of the representation.

        Returns:
            int: dimension of the representation
        """
        return 1

    def compute_rep(self, signatures):
        """
        Compute the sign representation from the signatures.

        Args:
            signatures (torch.tensor): (group.order, 1) tensor of signatures

        Returns:
            torch.tensor: (group.order, 1, 1) tensor of sign representations
        """
        rep = torch.zeros(self.order, 1, 1).cuda()
        rep[:, 0, 0] = signatures
        return rep

sign_rep = SignRepresentation([group.signatures], rep_params)


Computing trace tensor cube for sign representation
... loading from file


Next, we construct the natural representation. This isn't actually an irreducible representation, but is easy to construct. We can later decompose it into a direct sum of the trivial representation and the `standard` representation. The natural representation itself is $d=5$, with group elements mapping naturally to the 5x5 [permutation matrices](https://en.wikipedia.org/wiki/Permutation_matrix) (with a 1 in each row and column, and 0s elsewhere).

In [23]:
class NaturalRepresentation(Representation):
    """
    Compute the natural representation of the symmetric group.
    """
    def __init__(self, compute_rep_params, init_rep_params):
        """
        Initialise the natural representation. 

        Args:
            compute_rep_params (list): idx_to_perm function required to compute the representation
            init_rep_params (dict): standard group parameters needed by the representation, including index, order, multiplication_table, inverses, all_data
        """
        self.friendly_name = 'natural'
        super().__init__(compute_rep_params, **init_rep_params, irrep=False)
    
    def get_rep_dim(self):
        """
        Get the dimension of the representation.

        Returns:
            int: dimension of the representation
        """
        return self.index

    def compute_rep(self, idx_to_perm):
        """
        Compute the natural representation by directly computing permutation matrices

        Args:
            idx_to_perm (function): Function that takes an index and returns the corresponding permutation object

        Returns:
            torch.tensor: (group.order, group.index, group.index) tensor of permutation matrices for each group element
        """
        idx = list(np.linspace(0, self.index-1, self.index))
        rep = torch.zeros(self.order, self.index, self.index).cuda()
        for x in range(self.order):
            rep[x, idx, idx_to_perm(x)(idx)] = 1
        return rep

natural_rep = NaturalRepresentation([group.idx_to_perm], rep_params)

/tmp/ipykernel_48175/3779721982.py:38: DeprecationWarning:

an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.



In [24]:
class StandardRepresentation(Representation):
    """
    Generate the standard representation of the symmetric group.

    """
    def __init__(self, compute_rep_params, init_rep_params):
        """
        Initialise the standard representation. 

        Args:
            compute_rep_params (list): list containing natural_reps object necessary to calculate the standard representation
            init_rep_params (dict): standard group parameters needed by the representation, including index, order, multiplication_table, inverses, all_data
        """
        self.friendly_name = 'standard'
        super().__init__(compute_rep_params, **init_rep_params)

    def get_rep_dim(self):
        """
        Get the dimension of the representation.

        Returns:
            int: dimension of the representation
        """
        return self.index-1
    
    def compute_rep(self, natural_reps):
        """
        Compute the standard representation from the natural representation.

        Args:
            natural_reps (torch.tensor): (group.order, group.index, group.index) tensor of natural representations (permutation matrices)

        Returns:
            torch.tensor: (group.order, group.index-1, group.index-1) tensor of standard representations
        """
        rep = []
        basis_transform = torch.zeros(self.index, self.index).cuda()
        for i in range(self.index-1):
            basis_transform[i, i] = 1
            basis_transform[i, i+1] = -1
        basis_transform[self.index-1, self.index-1] = 1 #to make the transform non singular
        for x in natural_reps:
            temp = basis_transform @ x @ basis_transform.inverse()
            rep.append(temp[:self.index-1, :self.index-1])
        rep = torch.stack(rep, dim=0).cuda()
        return rep  

standard_rep = StandardRepresentation([natural_rep.rep], rep_params)


Computing trace tensor cube for standard representation
... loading from file


In [25]:
class StandardSignRepresentation(Representation):
    def __init__(self, compute_rep_params, init_rep_params):
        """
        Initialise the tensor product of the standard and sign representation. 

        Args:
            compute_rep_params (list): list consisting of tensor of standard representation and signatures, required to compute the representation
            init_rep_params (dict): standard group parameters needed by the representation, including index, order, multiplication_table, inverses, all_data
        """
        self.friendly_name = 'standard_sign'
        super().__init__(compute_rep_params, **init_rep_params)

    def get_rep_dim(self):
        """
        Get the dimension of the representation.

        Returns:
            int: dimension of the representation
        """
        return self.index-1

    def compute_rep(self, standard_reps, signatures):
        """
        Compute the tensor product of the standard and sign representation.

        Args:
            standard_reps (torch.tensor): (group.order, group.index-1, group.index-1) tensor of standard representations
            signatures (torch.tensor): (group.order, 1) tensor of signatures

        Returns:
            torch.tensor: (group.order, group.index-1, group.index-1) tensor of standard_sign representations
        """
        rep = []
        for i in range(standard_reps.shape[0]):
            rep.append(signatures[i]*standard_reps[i])
        rep = torch.stack(rep, dim=0).cuda()
        return rep
    
standard_sign_rep = StandardSignRepresentation([standard_rep.rep, sign_rep.rep], rep_params)      

Computing trace tensor cube for standard_sign representation
... loading from file


In [26]:
class SymmetricRepresentationFromGenerators(Representation):
    def __init__(self, compute_rep_params, init_rep_params, name):
        """
        Initialise a representation of the symmetric group from a set of generators.

        Args:
            compute_rep_params (list): list consisting of the generators of the representation, sympy group object, and a function that maps indices to permutations
            init_rep_params (dict): standard group parameters needed by the representation, including index, order, multiplication_table, inverses, all_data
            name (str): name of the representation
        """
        self.friendly_name = name
        super().__init__(compute_rep_params, **init_rep_params)

    # TODO: make this less hacky
    def get_rep_dim(self):
        """
        Get the dimension of the representation.

        Returns:
            int: dimension of the representation
        """
        return list(self.compute_rep_params[0].values())[0].shape[0] # hacky way to get the dimension of the representation

    def compute_rep(self, generators, G, idx_to_perm):
        """
        Compute the representation from the generators.

        Args:
            generators (dict): dictionary of generators of the group along with their representations
            G (sympy group object): group object
            idx_to_perm (function): function that maps indices to permutations

        Returns:
            torch.tensor: (group.order, dim, dim) tensor of arbitrary representations
        """
        rep = torch.zeros(self.order, self.dim, self.dim).cuda()
        for i in range(self.order):
            generator_product = G.generator_product(idx_to_perm(i), original=True)
            result = torch.eye(self.dim).float()
            for g in generator_product:
                result = result @ generators[g]
            rep[i] = result
        return rep.cuda()


s5_5d_a_generators = {}
s5_5d_a_generators[Permutation(0, 1, 2, 3, 4)] = torch.tensor([
    [ 1, -1, -1,  1,  0],
    [ 0, -1, -1,  0,  1],
    [ 1, -1,  0,  0,  0],
    [ 0, -1,  0,  0,  0],
    [ 1, -1, -1,  0,  0]
]).float()
s5_5d_a_generators[Permutation(4, 3, 2, 1, 0)] = s5_5d_a_generators[Permutation(0, 1, 2, 3, 4)].inverse()
s5_5d_a_generators[Permutation(4)(0,1)] = torch.tensor([
    [ 0,  0, -1,  0,  0],
    [ 0,  0,  0, -1,  0],
    [-1,  0,  0,  0,  0],
    [ 0, -1,  0,  0,  0],
    [ 0,  0,  0,  0, -1]
]).float()
s5_5d_a_rep = SymmetricRepresentationFromGenerators([s5_5d_a_generators, group.G, group.idx_to_perm], rep_params, 's5_5d_a')

# 3,2 specht
s5_5d_b_generators = {}
s5_5d_b_generators[Permutation(0, 1, 2, 3, 4)] = torch.tensor([
    [-1,  1, -1,  0,  0],
    [ 0,  0,  0,  1, -1],
    [ 0,  0,  1,  0, -1],
    [ 1,  0,  0,  0,  0],
    [ 1,  0,  1,  0,  0]
]).float()
s5_5d_b_generators[Permutation(4, 3, 2, 1, 0)] = s5_5d_b_generators[Permutation(0, 1, 2, 3, 4)].inverse()
s5_5d_b_generators[Permutation(4)(0,1)] = torch.tensor([
    [ 1,  0,  0,  0,  0],
    [ 0,  0,  0, -1,  0],
    [-1,  0,  0,  0, -1],
    [ 0, -1,  0,  0,  0],
    [-1,  0, -1,  0,  0]
]).float()
s5_5d_b_rep = SymmetricRepresentationFromGenerators([s5_5d_b_generators, group.G, group.idx_to_perm], rep_params, 's5_5d_b')

# 3,1,1 specht
s5_6d_generators = {}
s5_6d_generators[Permutation(0, 1, 2, 3, 4)] = torch.tensor([
    [ 1, -1,  1,  0,  0,  0],
    [ 1,  0,  0, -1,  1,  0],
    [ 0,  1,  0, -1,  0,  1],
    [ 1,  0,  0,  0,  0,  0],
    [ 0,  1,  0,  0,  0,  0],
    [ 0,  0,  0,  1,  0,  0]
]).float()
s5_6d_generators[Permutation(4, 3, 2, 1, 0)] = s5_6d_generators[Permutation(0, 1, 2, 3, 4)].inverse()
s5_6d_generators[Permutation(4)(0,1)] = torch.tensor([
    [-1,  0,  0,  0,  0,  0],
    [ 0,  0,  0, -1,  0,  0],
    [ 0,  0,  0,  0, -1,  0],
    [ 0, -1,  0,  0,  0,  0],
    [ 0,  0, -1,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  1]
]).float()
s5_6d_rep = SymmetricRepresentationFromGenerators([s5_6d_generators, group.G, group.idx_to_perm], rep_params, 's5_6d')


Computing trace tensor cube for s5_5d_a representation
... loading from file
Computing trace tensor cube for s5_5d_b representation
... loading from file
Computing trace tensor cube for s5_6d representation
... loading from file


In [27]:
group.irreps = {
    'trivial': trivial_rep,
    'sign': sign_rep,
    'standard': standard_rep,
    'standard_sign': standard_sign_rep,
    's5_5d_a': s5_5d_a_rep,
    's5_5d_b': s5_5d_b_rep,
    's5_6d': s5_6d_rep
}

Now we have all our representations loaded, let's inspect them...